<a href="https://colab.research.google.com/github/RusAl84/IntroML/blob/master/2_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Виды сверток



# Двумерные свертки
Мы уже знакомы с понятием свертки, но давайте углубимся в понимание ее работы, познакомимся с видами и параметрами сверток. Для наглядности, мы будем работать с **двумерными** свертками, но все сказанное здесь применимо к сверткам любой размерности.

Вспомним, как работают свертки. В отличии от полносвязного слоя, который умножает все входы на веса, в свертке мы умножаем на веса только избранные входы. Какие именно зависит от вида и параметров свертки. Веса свертки записываются в массив, **ядро** (kernel). В зависимости от размерности этого массива мы получим свертки разной размерности, одномерные, двумерные, трехмерные. Самые распространенные это двумерные свертки, они естественны для работы с двумерными массивами, которыми обычно представляются изображения.

В полносвязном слое нейронов, если мы поменяем местами входы (не забыв про веса), то это никак не повлияет на результат, однако в свертке, изменение порядка входов приведет к изменению результата - свертки чувствительны к **порядку** элементов.

Ниже показан двумерный массив 8x8 описывающий некоторое изображение в бинарных пикселях (1 - белый, 0 - черный), это наш вход.

![img](https://drive.google.com/uc?id=15kNAHVdEG-kb5-KS39p4BFNeOgdLYaIr)

Ядро свертки является двумерным массивом, размером 3*3:

$
kernel=
\begin{bmatrix}
     1          & 0      & 1     \\
     0          & 1    & 0     \\
     1          & 0    & 1
\end{bmatrix}
\\
$

Ниже показана анимация применения этого ядра к входному массиву:
- берется подмассив (оранжевый) размером 3*3 из входного массива,
- его значения умножаются поэлементно на значения из ядра, и все такие произведения складываются, это, по-сути, операция скалярного произведения векторов.
- получается одно число, элемент, результата (розовый).
- потом берется другой подмассив, смещая окошко выбора по горизонтали и вертикали и процесс повторяется. Результаты записываются в массив, номера элементов определяются смещением окошка выбора. Раз смещали окошко по двум координатам, то и результат - двумерный массив. Его называют **картой признаков** (feature map).

![img](https://drive.google.com/uc?id=1GMGRr9CFX-_zJ8pHdFfKtIAUDWVjobiy)
 <a href="http://cs231n.github.io/convolutional-networks/">ref</a></center>  


Здесь, используя ядро 3 * 3, применив его ко входу 5 * 5, получили результат размером 4 * 4. Посчитайте самостоятельно для ядер другого размера. Ядра не обязаны быть квадратными.

Вспомним также про параметры **набивки** (padding), когда мы добавляем фиктивные элементы к входному массиву, чтобы решулировать размер выхода и **сдвига** (stride), когда мы устанавливаем насколько элементов сдвигать окно выбора элементов входа. Несколько ядер могут быть объединены в **фильтр**, тогда результаты их складываются и ко всем элементам добавляется смещение (bias). Мы можем применять в одному и тому же входу несколько фильтров.

# Свертки в TensorFlow

Свертки реализованы во многих библиотеках, например в numpy, tensorflow, keras и других.  Давайте более детально познакомимся с некоторыми параметрами сверток.

Принята такая интерпретация размерностей массивов (в некоторых библиотеках мы можем устанавливать это сами) для работы с изображениями:

Для ядер:
- первые два измерения - пространственные, ширина и высота.
- третье измерение - каналы.
- четвертое измерение - фильтры.

Для входов и выходов:
- первое измерение - примеры, обучающие или тестовые.
- второе и третье - пространственные, ширина и высота.
- четвертое измерение - каналы. Как вы понимаете, если мы применяем более одного фильтра, то фильтры сворачивают каналы входа, а сами являются каналами выхода.

В Keras (tensorflow) за создание двумерной свертки отвечает функция [Conv2D](https://keras.io/api/layers/convolution_layers/convolution2d/)

```
tf.keras.layers.Conv2D(
    filters,
    kernel_size,
    strides=(1, 1),
    padding="valid",
    data_format=None,
    dilation_rate=(1, 1),
    groups=1,
    activation=None,
    use_bias=True,
    kernel_initializer="glorot_uniform",
    bias_initializer="zeros",
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,
    **kwargs
)
```
У нее много параметров, давайте посмотрим на них. Сделаем вход размера (4, 28, 28, 12) и посмотрим, какой получится выход при разных значениях параметров.



In [ ]:
import tensorflow as tf # подключаем библиотеку
input_shape = (4, 28, 28, 12) # размер входа
x = tf.random.normal(input_shape) # делаем вход нужного размера из случайных чисел
print(x.shape) # размер полученного массива

(4, 28, 28, 12)


- **filters**: целое число, устанавливающее число фильтров. Оно же означает число каналов выхода. Число ядер в каждом фильтре отределяется числом каналов входа. Это первый обязательный аргумент. Обязательным является и второй, kernel_size. Пробуйте менять первый аргумент и смотрите как изменяется выход.

In [ ]:
y = tf.keras.layers.Conv2D(10,3)(x) # делаем свертку с 10 фильтрами, каждый с ядрами 3*3.
print(y.shape) # размер полученного массива

(4, 26, 26, 10)


- **kernel_size**: целое число или пара чисел, определяющая размер ядра в фильтре (высота, ширина). Это обязательный аргумент. Если одно число, то ядро квадратное. Пробуйте разные размеры ядра. Предположите заранее размер выхода, а потом проверьте расчетом.

In [ ]:
y = tf.keras.layers.Conv2D(1,(2,3))(x) # делаем свертку с 1 фильтром, с ядром 2*3.
print(y.shape) # размер полученного массива

(4, 27, 26, 1)


- **strides:** сдвиг, целое число или пара чисел, определяющее на сколько элементов будет перемещаться окно выбора. Если одно число, то сдвиги по обеим осям одинаковые. По умолчанию (1,1). Пробуйте разные сдвиги. Предположите заранее размер выхода, а потом проверьте расчетом.

In [ ]:
y = tf.keras.layers.Conv2D(1,2, strides=(2,3))(x) # делаем свертку с 1 фильтром, с ядром 2*2, но сдвиг по одной оси 2, а по другой 3.
print(y.shape) # размер полученного массива

(4, 14, 9, 1)


- **padding**: набивка, принимает значения "valid" или "same". "valid" (по умолчанию) означает нет набивки. "same" означает, что набивка такая, что пространственная размерность выхода такая же как у входа. Проверьте оба варианта.

In [ ]:
y = tf.keras.layers.Conv2D(1,2, padding='same')(x) # делаем свертку с 1 фильтром, с ядром 2*2, но набивка 'same'.
print(y.shape) # размер полученного массива

(4, 28, 28, 1)


- **groups**: позволяет разбить каналы входа на указанное число групп, и использовать их как будто это отдельные входы со своими фильтрами. Полезно для более простой обработки многоканальных сверток. Пробуйте разные параметры.

Число каналов входа и число фильтров должно делиться на groups. Например, если вход 12 канальный, фильтров 6, а группа одна, то каждый фильтр будет принимать 12 канальный вход (значит в нем будет 12 ядер). Если групп 3, то в этом случае каждый из 6 фильтров будет принимать только 4 своих каналов. Размер выхода будет такой же, но число обучаемых параметров - другое. Не работает на CPU.

![img](https://images1.russianblogs.com/207/df/dfc8a0eea21789e834a3c7e1b6747f17.png )
![img](https://images2.russianblogs.com/387/aa/aaad0eee14c1ec46d4b2914737ed8ae3.png)   

In [ ]:
# Одна группа
layer=tf.keras.layers.Conv2D(6,2,groups=1)# делаем свертку с 6 фильтрами, с ядром 2*2, с одной группой (оно же по умолчанию).
y = layer(x)
print('1 group conv')
print('Kernel size: ',layer.trainable_variables[0].shape) # размер массива весов ядер (это свойство доступно после проведения расчетов, но не до)
print('Bias   size:', layer.trainable_variables[1].shape) # размер массива смещений ядер (это свойство доступно после проведения расчетов, но не до)
print('Output size:', y.shape) # размер полученного массива

# Три группы
layer=tf.keras.layers.Conv2D(6,2,groups=3) # делаем свертку с 6 фильтрами, с ядром 2*2, с тремя группами (значит каждому ядру достанется по 4 канала).
y = layer(x)
print()
print('3 group conv')
print('Kernel size: ',layer.trainable_variables[0].shape) # размер массива весов ядер (это свойство доступно после проведения расчетов, но не до)
print('Bias   size:', layer.trainable_variables[1].shape) # размер массива смещений ядер (это свойство доступно после проведения расчетов, но не до)
print('Output size:', y.shape) # размер полученного массива



1 group conv
Kernel size:  (2, 2, 12, 6)
Bias   size: (6,)
Output size: (4, 27, 27, 6)

3 group conv
Kernel size:  (2, 2, 4, 6)
Bias   size: (6,)
Output size: (4, 27, 27, 6)


- **activation**: можно указать функцию активации для слоя. По умолчанию не применяется.

Другие аргументы смотри в документации.

## Распределенная свертка
Во всех примерах сверток выше мы брали соседние элементы массива входов для умножения на веса ядер. Но почему? Нам никто не мешает брать эти элементы не подряд, а, скажем, через один. Так получается распределенная (dilated) свертка.

Обычная свертка:

![img](https://drive.google.com/uc?id=1b1g3EHRPBsFBXEFXyzJv5hjpBJMDOjpc)

Распределенная свертка:

![img](https://drive.google.com/uc?id=1VQ_GuYh32RpIBq1N1y1rvPbDQ5Shtz6c)

Смысл ее в том, что она охватывает гораздо больший подмассив входов. Согласитесь, ведь не всегда соседние пиксели важны, и нет смысла их все обрабатывать. На картинке ниже показаны свертки с частотой один (обычная), два и четыре, (_примененные последовательно к результатам предыдущей_), сравните размер их окошек. Такие свертки помогают проще работать с изображениями разного масштаба.

![img](https://drive.google.com/uc?id=1GYMjAuUe_TZJEboRP0ZB9ZvEogEd2_TG)



Аргумент
- **dilation_rate**: показвает частоту распределения свертки, целое число, или пара чисел. В текущей реализации не совместимо со stride не равным 1.   

In [ ]:
layer=tf.keras.layers.Conv2D(1,3,dilation_rate=1)# делаем свертку с 1 фильтром, с ядром 3*3, с частотой 1 (оно же по умолчанию).
y = layer(x)
print(y.shape) # размер полученного массива

layer=tf.keras.layers.Conv2D(1,3,dilation_rate=4)# делаем свертку с 1 фильтром, с ядром 3*3, с частотой 4.
y = layer(x)
print(y.shape) # размер полученного массива



(4, 26, 26, 1)
(4, 20, 20, 1)


# Транспонированная свертка
Как мы видели, пространственный размер выхода обычной свертки меньше размера входа. Сдвиг позволяет его уменьшить еще существеннее. Но если нам надо наоборот, увеличить размер выхода? На помощь придет **транспонированная свертка**. В одном варианте объединили идеи набивки и распределения.

![img](https://drive.google.com/uc?id=1M00SUu0SQ3XMXjEEolxGxKOLmRMzGVKA)

Элементы входа (синие) распределяются, между ними добавляются пустые элементы (нули), набивка (пунктир). И уже к этому массиву применяется свертка. Как видите выход (зеленый) получился размером больше. Часто транспонированную свертку называют (неправильно) **разверткой** (deconvolution). Правильно говорить, что эта операция похожа на развертку. Реальные развертки (развертка - обратная операция по отношению к свертке) гораздо сложнее, но, к счастью, они нам и не нужны для многих задач.


Вариант транспонированной свертки [подробности](https://medium.com/apache-mxnet/transposed-convolutions-explained-with-ms-excel-52d13030c7e8).
Как вообще из одного числа сделать 9? Скопировать!

Давайте будем копировать (размножать) входной элемент заданное количество раз, а потом сложим значения, полученные от копирования разных входов.

![img](https://miro.medium.com/max/977/1*kOThnLR8Fge_AJcHrkR3dg.gif)

Чтобы эта опреация была обучаемая, возьмем ядро и будем умножать на него (поэлементно) размноженный вход.
Если посмотреть, какие элементы входа влияют на конкретный элемент выхода

![img](https://miro.medium.com/max/3349/1*yoQ62ckovnGYV2vSIq9q4g.gif)

и с какими весами они идут, то увидим, что ядро симметрично отразилось (транспонировалось).  

![img](https://miro.medium.com/max/977/1*HnxnJDq-IgsSS0q3Lut4xA.gif)

В Keras за такую свертку отвечает  [Conv2DTranspose](https://keras.io/api/layers/convolution_layers/convolution2d_transpose/), аргументы которой (это ведь тоже свертка) очень похожи на Conv2D.



In [ ]:
layer=tf.keras.layers.Conv2D(1,3,strides=2)# делаем свертку с 1 фильтром, с ядром 3*3, со сдвигом 2.
y = layer(x)
print(y.shape) # размер полученного массива

layer=tf.keras.layers.Conv2DTranspose(1,3,strides=2)# делаем транспонированную свертку с 1 фильтром, с ядром 3*3, со сдвигом 2, частотой 1
y = layer(x)
print(y.shape) # размер полученного массива

layer=tf.keras.layers.Conv2DTranspose(1,3,dilation_rate=2)# делаем транспонированную свертку с 1 фильтром, с ядром 3*3, со сдвигом 1, частотой 2
y = layer(x)
print(y.shape) # размер полученного массива


(4, 13, 13, 1)
(4, 57, 57, 1)
(4, 32, 32, 1)


# Разделимые свертки
В многослойном персетпроне мы видели, что применять подряд два слоя с линейной функцией активации бессмысленно, они эквивалентны одному. А что в свертках? Свертки различаются между собой не только весами ядер, но и тем, к каким именно элементам они применяются. Например, можно применить свертки к пространственным измерениям, а потом к каналам или наоборот. И получатся разные результаты.

Такие свертки назвали разделимые (divided или separable) и они нашли очень большое применение из-за их вычислительной эффективности.

Можно придумать разные варианты таких сверток, реализован вариант [SeparableConv2D](https://keras.io/api/layers/convolution_layers/separable_convolution2d/) в котором сначала каждый канал сворачивается со своим ядром, а результат, сворачивается уже вдоль каналов сверткой 1 * 1. Это не тоже самое что использовать несколько фильтров. Сравните два варианта, обычная свертка с 6 фильтрами и разделимая. У обычной каждый из 6 фильтров имеет размер ядер 2 * 2 * 12. В разделимой из 6 фильтров есть 12 общих пространственных ядер 2 * 2  и 6 отдельных ядер размером 12 * 1, по одному для каждого фильтра. Сравните число весов, причем размер результата такой же. Меньше весов - быстрей обучать. Разделимые свертки используют, когда надо считать быстро, например на мобильных устройствах.         

![img](https://drive.google.com/uc?id=1CUxOExwLDMyxDCRTIbP-8ltw7e2sgCk-)


In [ ]:
# обычная свертка с 6 фильтрами
layer=tf.keras.layers.Conv2D(6,2)
y = layer(x)
print('Conv2D')
print('Kernel size        ',layer.trainable_variables[0].shape) # размер массива весов пространственных ядер
print('Bias   size        ',layer.trainable_variables[1].shape) # размер массива смещений
print('Output size        ',y.shape)

# разделимая свертка с 6 фильтрами
layer=tf.keras.layers.SeparableConv2D(6,2)
y = layer(x)
print('SeparableConv2D')
print('Spatial kernel size',layer.trainable_variables[0].shape) # размер массива весов пространственных ядер
print('Channel kernel size',layer.trainable_variables[1].shape) # размер массива весов канальных ядер
print('Bias           size',layer.trainable_variables[2].shape) # размер массива смещений ядер
print('Output         size',y.shape)



Conv2D
Kernel size         (2, 2, 12, 6)
Bias   size         (6,)
Output size         (4, 27, 27, 6)
SeparableConv2D
Spatial kernel size (2, 2, 12, 1)
Channel kernel size (1, 1, 12, 6)
Bias           size (6,)
Output         size (4, 27, 27, 6)


# Деформируемые свертки
Свертки не обязаны быть прямоугольными.
Например, существуют деформируемые свертки, в которых номер элемента входа,который берется в свертку, сам рассчитывается за счет обучения еще одного параметра - отклонения (offset).   

![img](https://machinelearningmastery.ru/img/0-788232-675235.png)

# Задания
Изменяйте параметры сверток, смотрите как это влияет на размерность выхода и количество параметров.

# Ссылки
Использованы и адаптированы материалы:


https://colab.research.google.com/github/Gurubux/CognitiveClass-DL/blob/master/2_Deep_Learning_with_TensorFlow/DL_CC_2_2_CNN/2.1-Review-Understanding_Convolutions.ipynb

https://datascience.stackexchange.com/questions/6107/what-are-deconvolutional-layers

https://medium.com/@zurister/depth-wise-convolution-and-depth-wise-separable-convolution-37346565d4ec

https://keras.io/

https://machinelearningmastery.ru/types-of-convolution-kernels-simplified-f040cb307c37/

